In [1]:
%load_ext autoreload
%autoreload 2
# !pip install ipywidgets
import sys
sys.path.append("/home/boittier/Documents/phd/ff_energy")

from ff_energy.structure import Structure
from ff_energy.job import Job
from ff_energy.jobmaker import get_structures_pdbs, JobMaker
from ff_energy.plot import plot_energy_MSE
from ff_energy.configmaker import *

from pathlib import Path
import pandas as pd

# s = Structure("/home/boittier/charmm/mix3/jobs/pdbs/mix0.pdb")

def MakeJob(name, ConfigMaker):
    structures, pdbs = get_structures_pdbs(
        Path(ConfigMaker.pdbs),
        atom_types=atom_types
    ) 
    return JobMaker(name, pdbs, structures, ConfigMaker.make().__dict__)

atom_types = {
              ("LIG", "O"): "OG311",
              ("LIG", "C"): "CG331",
              ("LIG", "H1"): "HGP1",
              ("LIG", "H2"): "HGA3",
              ("LIG", "H3"): "HGA3",
              ("LIG", "H4"): "HGA3",
              ("TIP3", "OH2"): "OT",
              ("TIP3", "H1"): "HT",
              ("TIP3", "H2"): "HT",
              # ("LIG", "O"): "OT",
              # ("LIG", "H1"): "HT",
              # ("LIG", "H"): "HT",
              # ("LIG", "H2"): "HT",
              }




m_basis: avdz
m_method: hf
c_files: ['pbe0_dz.pc']
c_dcm_command: open unit 11 card read name p 
DCM IUDCM 11 TSHIFT XYZ 15
m_nproc: 8
m_memory: 150
m_queue: short
chmpath: /home/boittier/dev-release-dcm/build/cmake/charmm
modules: module load cmake/cmake-3.23.0-gcc-11.2.0-openmpi-4.1.3



In [2]:
CMS = []
for system in system_names:
    for theory in THEORY.keys():
        print(system, theory)
        cm = ConfigMaker(theory, system, "pc")
        # print(cm.pdbs)
        print(cm.make().kwargs)
        print()
        CMS.append(cm)

water_cluster pbe0dz
<bound method Config.kwargs of m_basis: avdz
m_method: hf
c_files: ['pbe0_dz.pc']
c_dcm_command: open unit 11 card read name p 
DCM IUDCM 11 TSHIFT XYZ 15
m_nproc: 8
m_memory: 150
m_queue: short
chmpath: /home/boittier/dev-release-dcm/build/cmake/charmm
modules: module load cmake/cmake-3.23.0-gcc-11.2.0-openmpi-4.1.3
>

water_dimer pbe0dz
<bound method Config.kwargs of m_basis: avdz
m_method: hf
c_files: ['pbe0_dz.pc']
c_dcm_command: open unit 11 card read name p 
DCM IUDCM 11 TSHIFT XYZ 15
m_nproc: 8
m_memory: 150
m_queue: short
chmpath: /home/boittier/dev-release-dcm/build/cmake/charmm
modules: module load cmake/cmake-3.23.0-gcc-11.2.0-openmpi-4.1.3
>

methanol_cluster pbe0dz
<bound method Config.kwargs of m_basis: avdz
m_method: hf
c_files: ['meoh_pbe0dz.pc']
c_dcm_command: open unit 11 card read name m 
DCM IUDCM 11 TSHIFT XYZ 15
m_nproc: 8
m_memory: 150
m_queue: short
chmpath: /home/boittier/dev-release-dcm/build/cmake/charmm
modules: module load cmake/cmake-3

In [3]:
jm = MakeJob("meoh_clusters/pbe0dz", CMS[2])
jm

In [4]:
HOMEDIR = "/home/boittier/pcbach"
jm.make_molpro(HOMEDIR)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 69.57it/s]


In [5]:
from ff_energy.slurm import SlurmJobHandler

shj = SlurmJobHandler(max_jobs=100)

print("Running jobs: ", shj.get_running_jobs())

for js in jm.get_cluster_jobs(HOMEDIR):
    shj.add_job(js)

print("Jobs: ", len(jm.get_cluster_jobs(HOMEDIR)))

shj.submit_jobs()

Running jobs:  91
Jobs:  7
Submitted job /home/boittier/pcbach/meoh_clusters/pbe0dz/meoh99/cluster/meoh99.sh to Slurm scheduler
Submitted job /home/boittier/pcbach/meoh_clusters/pbe0dz/meoh96/cluster/meoh96.sh to Slurm scheduler
Submitted job /home/boittier/pcbach/meoh_clusters/pbe0dz/meoh44/cluster/meoh44.sh to Slurm scheduler
Submitted job /home/boittier/pcbach/meoh_clusters/pbe0dz/meoh90/cluster/meoh90.sh to Slurm scheduler
Submitted job /home/boittier/pcbach/meoh_clusters/pbe0dz/meoh62/cluster/meoh62.sh to Slurm scheduler
Submitted job /home/boittier/pcbach/meoh_clusters/pbe0dz/meoh49/cluster/meoh49.sh to Slurm scheduler
Submitted job /home/boittier/pcbach/meoh_clusters/pbe0dz/meoh67/cluster/meoh67.sh to Slurm scheduler


In [9]:
jm.get_cluster_jobs(HOMEDIR)

['/home/boittier/pcbach/meoh_clusters/pbe0dz/mix303/cluster/mix303.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix42/cluster/mix42.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix3/cluster/mix3.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix150/cluster/mix150.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix588/cluster/mix588.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix123/cluster/mix123.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix538/cluster/mix538.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix410/cluster/mix410.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix613/cluster/mix613.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix668/cluster/mix668.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix723/cluster/mix723.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix360/cluster/mix360.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix878/cluster/mix878.sh',
 '/home/boittier/pcbach/meoh_clusters/pbe0dz/mix746/cluster/mix746.sh'